In [2]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [3]:
from string import translate, maketrans

In [1]:
%store -r numerosKalaba
# numerosKalaba=[5]
numeroKalaba="20-K%d"%numerosKalaba[0]
print numeroKalaba

20-K4


In [4]:
# -*- coding: utf8 -*-
home = expanduser("~")
serie=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%numeroKalaba

nomsPropres=["Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["kaleb","maira","vjolet","nabil","katiSa","nikol"]

with open(serie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.load(stream)

In [5]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [6]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(prevStems)

In [7]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [8]:
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)

In [9]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [24]:
consonnes

'ptkbdgmnNfTsSvDzZrljw'

In [77]:
from numpy.random import choice

consonnesFreq=[10,10,10,10,10,10,8,8,6,7,7,7,7,5,5,5,5,8,8,3,3]
consonnesTot=sum(consonnesFreq)
consonnesProb=[float(k)/consonnesTot for k in consonnesFreq]


In [78]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in "iae":
            cibles.append(p)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=phonology["apophonies"][result[cible]]
        return "".join(result)
    else:
        return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [79]:
operations=[devoiserFinale,devoiserFinale,voiserVCV,voiserVCV,changerLongueur,changerLongueur,apophonie,metathese]

In [80]:
def randomRadical(radical):
    result=""
    for letter in radical:
        if letter in voyelles:
            result+=rd.choice(voyelles)
        else:
            result+=choice(list(consonnes), p=consonnesProb)
    print "random",result
    return result

def modifierRadical(radical,dist=[0,1,2,2,3,3,3,4,5]):
    nOperations=rd.choice(dist)
    while nOperations>0:
        operation=rd.choice(operations)
        print nOperations,operation
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    print "modif",radical
    return radical

In [109]:
modifierRadical(randomRadical(u"kaleb"),dist=[0,0,0,1,1,2])

random dekebu
modif dekebu


'dekebu'

In [14]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(randomRadical(prevLexique[mot]),dist=[0,0,0,1,1,2])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

random Tatare
1 <function apophonie at 0x104a30488>
modif Tature
random riril
modif riril
random viDib
modif viDib
random wugir
1 <function voiserVCV at 0x104a30230>
1 <function apophonie at 0x104a30488>
modif wugar
random veSas
2 <function voiserVCV at 0x104a30230>
1 <function changerLongueur at 0x104a30578>
modif veZasa
random pimiT
modif pimiT
random Zetufa
modif Zetufa
random fomil
1 <function apophonie at 0x104a30488>
modif fomal
random rupilu
modif rupilu
random gefisu
1 <function voiserVCV at 0x104a30230>
modif gevisu
random guval
2 <function apophonie at 0x104a30488>
1 <function voiserVCV at 0x104a30230>
1 <function changerLongueur at 0x104a30578>
1 <function metathese at 0x104a30668>
1 <function devoiserFinale at 0x104724e60>
1 <function changerLongueur at 0x104a30578>
1 <function voiserVCV at 0x104a30230>
1 <function changerLongueur at 0x104a30578>
1 <function devoiserFinale at 0x104724e60>
1 <function metathese at 0x104a30668>
1 <function metathese at 0x104a30668>
modif gulu

In [15]:
modifierRadical("Tonif")

4 <function voiserVCV at 0x104a30230>
4 <function changerLongueur at 0x104a30578>
4 <function changerLongueur at 0x104a30578>
4 <function changerLongueur at 0x104a30578>
4 <function apophonie at 0x104a30488>
3 <function devoiserFinale at 0x104724e60>
3 <function devoiserFinale at 0x104724e60>
3 <function changerLongueur at 0x104a30578>
3 <function changerLongueur at 0x104a30578>
3 <function voiserVCV at 0x104a30230>
3 <function apophonie at 0x104a30488>
2 <function metathese at 0x104a30668>
2 <function changerLongueur at 0x104a30578>
2 <function devoiserFinale at 0x104724e60>
2 <function devoiserFinale at 0x104724e60>
2 <function metathese at 0x104a30668>
1 <function changerLongueur at 0x104a30578>
modif Tofuno


'Tofuno'

In [16]:
racineChanges

{'Davin': 'fobik',
 'Detip': 'Nafus',
 'Dipud': 'sanug',
 'Dodav': 'wobek',
 'Dop': 'Nuwi',
 'Duluv': 'Dazak',
 'Dutej': 'Zudoka',
 'Natuzu': 'Zetufa',
 'Nebot': 'budug',
 'Newuze': 'roNimu',
 'Nov': 'lof',
 'NuNod': 'fibuN',
 'Nusaw': 'kofop',
 'Nutul': 'zedis',
 'Sagof': 'veziN',
 'Sezag': 'kikow',
 'Sikele': 'posuso',
 'Siroz': 'veZasa',
 'Sizez': 'turaj',
 'Sobil': 'nuveb',
 'Sokos': 'zabep',
 'Sol': 'put',
 'Somun': 'pisow',
 'Suzun': 'goloS',
 'TewaD': 'womaD',
 'Tibuj': 'vozilo',
 'Todav': 'rubaf',
 'Tokib': 'zaser',
 'Tomiva': 'wagop',
 'Tozan': 'nunusa',
 'Zenib': 'dazaT',
 'Zeres': 'wugar',
 'Zetev': 'fomal',
 'ZiwoDo': 'navuwo',
 'ZuNup': 'witoj',
 'b': 'to',
 'bo': 'no',
 'depiru': 'sijid',
 'dilik': 'mibaz',
 'dupag': 'TiwuNo',
 'fajam': 'robaZ',
 'fanot': 'roDuve',
 'feTet': 'sorawu',
 'felez': 'Noriw',
 'fiSel': 'SubaS',
 'fiTed': 'guluv',
 'fiZob': 'Nopot',
 'fisoZi': 'rupilu',
 'fiweZ': 'pavuf',
 'fuSob': 'tinig',
 'ful': 'taje',
 'gapok': 'tuSul',
 'gejufe': 'gevisu',

In [17]:
newStems=replace_keys(prevStems,racineChanges)

In [18]:
with open(serie+'randomStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [19]:
with open(serie+"randomStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [20]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  A1:
    Nopot: [trois]
    TaZug: [bas, basse, basses]
    Tumod: [suivant, suivants, suivante, suivantes]
    dugul: [gros, grosse, grosses]
    nunusa: [rouge, rouges]
    rubaf: [effrayé, effrayés, effrayée, effrayées]
    rutes: [différent, différents, différente, différentes]
    tinig: [courageux, courageuse, courageuses]
    zabep: [noir, noire, noirs, noires]
    zubug: [grand, grande, grands, grandes]
  A2:
    Noriw: [terrible, terribles]
    Tadet: [maigre, maigres]
    TiwuNo: [jaune, jaunes]
    furop: [quatre]
    kikow: [profond, profonds, profonde, profondes]
    pavub: [blanc, blanche, blancs, blanches]
    roDuve: [petit, petite, petits, petites]
    sorawu: [furieux, furieuse, furieuses]
    vepip: [blessé, blessés, blessée, blessées]
DET:
  f: [DEF, le, la, les, l]
  ji: [IND, un, une, des, dE]
  to: [DEM, ce, cet, cette, ces]
NOM:
  N1:
    N11:
      A:
        Tibiw: [forêt, forêts]
        daNafi: [balai, balais]
        sopib: [coup, coups]
        turaj

In [21]:
with open(serie+'randomStems.yaml', 'w') as output:
    output.write(yamlText)